In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
from google.cloud import aiplatform as vertex

In [4]:
PROJECT_ID = 'mitochondrion-project-344303'
EXPERIMENT_NAME = 'iris-xgboost'
REGION = 'asia-southeast1'

In [5]:
vertex.init(
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, location=REGION
)

In [6]:
X = iris.loc[:, 'sepal_length':'petal_width'].values
y = iris['species'].values

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [9]:
import xgboost as xgb

/home/mitbal/miniconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
max_depths = [5, 10, 15]
learning_rates = [0.3, 0.6, 0.9]

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
counter = 0
for depth in max_depths:
    for rate in learning_rates:

        vertex.start_run(f'run-{counter}')
        counter += 1
        
        xgb_model = xgb.XGBClassifier(objective='multi:softprob', random_state=41,
            learning_rate=rate, max_depth=depth, eval_metric='mlogloss')

        vertex.log_params({
            'learning_rate': rate,
            'max_depth': depth
        })

        xgb_model.fit(X=X_train, y=y_train)

        train_acc = np.sum(xgb_model.predict(X_train) == y_train) / len(y_train)
        test_acc = np.sum(xgb_model.predict(X_test) == y_test) / len(y_test)

        vertex.log_metrics({
            'train_acc': train_acc,
            'test_acc': test_acc
        })

        vertex.end_run()

Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-0 to Experiment: iris-xgboost
Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-1 to Experiment: iris-xgboost
Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-2 to Experiment: iris-xgboost
Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-3 to Experiment: iris-xgboost
Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-4 to Experiment: iris-xgboost
Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-5 to Experiment: iris-xgboost
Associating projects/832137092875/locations/asia-southeast1/metadataStores/default/contexts/iris-xgboost-run-6 to Experiment: iris-xgboost
Associating projects/832137